In [7]:
import sys
sys.path.append("../Scripts/")
import preprocess
import datasets

In [1]:
import os

train_images_path = "../Data/Train/"
image_folders = os.listdir(train_images_path)

for folder in image_folders:
    image_files = os.listdir(train_images_path+folder+"/")
    i = 0
    for image_file in image_files:
        ## os.rename(train_images_path+folder+"/"+image_file, train_images_path+folder+"/"+ folder + "_" + str(i) + ".jpg")
        i +=1

In [85]:
import os

# organize images for further processing
test_images_path = "../Data/Test/"
image_folders = os.listdir(test_images_path)

for folder in image_folders:
    image_files = os.listdir(test_images_path+folder+"/")
    i = 0
    for image_file in image_files:
        os.rename(test_images_path+folder+"/"+image_file, test_images_path+folder+"/"+ folder + "_" + str(i) + ".jpg")
        i +=1

In [21]:
from preprocess import create_csv_dataset
test_images_path = "../Data/Train/"
csv_name = "train_data"
csv_path = "../Data/"
create_csv_dataset(test_images_path, csv_name, csv_path)

In [26]:

test_images_path = "../Data/Test/"
csv_name = "test_data"
csv_path = "../Data/"
create_csv_dataset(test_images_path, csv_name, csv_path)